In [ ]:
from tensorflow.keras.models import load_model
import os, glob
from datetime import datetime as dt
import numpy as np
import multiprocessing as mp
import tensorflow.keras as keras
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from grad_cam import *

## Load data files for which to get heatmaps

In [ ]:
## Load and preprocess one case
def load_file(path):
    
    #Load case
    data = np.load(path)['arr_0'][0:None:3, 0:None:3, :]
        
    #Standardise fields
    for field in range(data.shape[-1]):
        avg_ = np.mean(data[:,:,field])
        std_ = np.std(data[:,:,field])
        data[:,:,field] = (data[:,:,field] - avg_) / std_ 

    #Obtain label
    name = path.replace(".npz", "").split("/")[-1]
    if "no" in name:
        cat, start_lat, end_lat, start_lon, end_lon, date = name.split("_")
        label = 0
    else:
        cat, press, wind, start_lat, end_lat, start_lon, end_lon, date = name.split("_")
        if int(cat) > 0:
            label = 1
        else:
            label = 0

    #Return preprocessed case and its label
    return data, label

#Load a number of files
def load_data(paths):
    
    #Open first file in list to get shape of cases
    dummy_file = np.load(paths[0])['arr_0'][0:None:3, 0:None:3, :]
    
    #Create array to hold all the preprocessed cases
    data = np.zeros((len(paths), dummy_file.shape[0], dummy_file.shape[1], dummy_file.shape[2]))
    
    #Create array to hold all the labels for the corresponding cases
    labels = np.zeros(len(paths))
    
    #Load files in parallel
    num_cores = np.min([len(paths), int(0.5*mp.cpu_count())])
    pool = mp.Pool(num_cores)
    res = list(pool.imap(load_file, paths))
    for i in range(len(res)):
        data[i, :, :, :] = res[i][0]
        labels[i] = res[i][1]
    pool.close()
        
    #Return cases and thier labels
    return data, labels

## Get list of all files available

In [ ]:
all_files = sorted(glob.glob(os.path.join("/work/scratch/dg/final_data_paper_1", "*")))

## Get files we are interested in 

In [ ]:
paths = []
for path in all_files:
    name = path.split("/")[-1]
    
    if "no" in name:
        pass
    else:
        cat, press, wind, start_lat, end_lat, start_lon, end_lon, date = name.replace(".npz", "").split("_")
        year = int(date[:4])
        month = int(date[4:6])
        day = int(date[6:8])
        hour = int(date[8:])
        date_time = dt(year, month, day, hour)
        if int(start_lat) == 0 and int(end_lat) == 60 and int(start_lon) == 260 and date_time >= dt(2005, 8, 28, 18) and date_time <= dt(2005, 8, 29, 0):
            paths.append(path)
        else:
            pass

## Load Model to generate heatmap for

In [ ]:
model = tf.keras.models.load_model("/home/users/dgalea/papers/paper1/code/final_data/Section 5 - Discussion/D - Model Generalization/gg.h5")

## Get model predictions

In [ ]:
data, labels = load_data(paths)
preds = model.predict(data)

## Compute and Save Heatmaps

In [ ]:
variables = ["MSLP", "10m wind speed", "850hPa Vorticity", "700hPa Vorticity", "600hPa Vorticity"]
heatmap = get_heatmap(model, "conv2d_3", data[0])    
name = paths[0].replace(".npz", "").split("/")[-1]
plot_heatmap(heatmap, data[0], preds[0], variables, name, save=True, show=True)